In [58]:
import pystac_client
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from itertools import compress
import pandas as pd
from shapely import wkt
import geopandas as gpd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# opening STAC catalog

catalog = pystac_client.Client.open(
    #"https://storage.googleapis.com/dgds-data-public/coclico/coclico-stac/catalog.json" # cloud API
    r"p:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata\current\catalog.json" # P drive API 
)
#catalog

In [59]:
href_dr = catalog.get_child("shore_mon_drivers").assets["data"].href
ds_dr = xr.open_zarr(href_dr)
ds_dr

<xarray.Dataset>
Dimensions:                (stations: 57924)
Coordinates:
    continent              (stations) |S13 dask.array<chunksize=(14481,), meta=np.ndarray>
    country                (stations) |S27 dask.array<chunksize=(14481,), meta=np.ndarray>
    country_id             (stations) |S7 dask.array<chunksize=(28962,), meta=np.ndarray>
    hotspot_id             (stations) |S31 dask.array<chunksize=(14481,), meta=np.ndarray>
    lat                    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    lon                    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    transect_geom          (stations) |S57 dask.array<chunksize=(7241,), meta=np.ndarray>
    transect_id            (stations) |S15 dask.array<chunksize=(14481,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    changerate             (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    date_nourishment       (stations) |S30 dask.array<chunksize=(14481,), meta=np.ndarray>
    ldb_type               (stations) |S14 dask.array<chunksize=(14481,), meta=np.ndarray>
    littoraldb_id_conf     (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    nourishment_id_conf    (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    reclamation_id_conf    (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    seasonal_displacement  (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    seasonal_id_conf       (stations) |S7 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_max_seasonal_sp      (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_min_seasonal_sp      (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_recl_construction    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Van der Heijden, D., Luijendijk, A. & Kras, E.
    CITATION:            van der Heijden, D. (2022). Global identification an...
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Shoreline Monitor Drivers dataset aims to identi...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'shoreline dynamics', 'big dat...
    TEMPORAL_EXTENT:     ['1984-01-01T00:00:00Z', '2022-01-01T00:00:00Z']
    TITLE:               The Shoreline Monitor Drivers
    TITLE_ABBREVIATION:  shore_mon_drivers
    UNITS:

In [60]:
country = [b'Portugal']
indices = np.where(ds_dr['country'].isin(country))[0]
df_port = ds_dr.isel(stations = indices).to_dataframe().reset_index()

decode_cols = ['seasonal_id_conf', 'littoraldb_id_conf', 'nourishment_id_conf', 'reclamation_id_conf', 'transect_id']
for dc in decode_cols:
    df_port[dc] = df_port[dc].str.decode("utf-8")




# Add a fake data point with the gray category and null location
fake_data = pd.DataFrame({'littoraldb_id_conf': ['medium'], 'lat': [None], 'lon': [None]})
df_nl = pd.concat([df_port, fake_data])

label_colors = {'low': 'red', 'medium':'orange', 'high': 'green'}
order = list(label_colors.keys())

fig = px.scatter_mapbox(df_port, 
                    lat='lat', 
                    lon = 'lon',
                    color='littoraldb_id_conf',
                    color_discrete_map= label_colors,
                    hover_data= ['transect_id'],
                    category_orders= {'littoraldb_id_conf': order},
                    center={'lat': df_port['lat'].mean(), 'lon': df_port['lon'].mean()},
                    mapbox_style='carto-positron', 
                    zoom=6.5,
                    width = 800,
                    height = 1000)

fig.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Littoral Drift Barrier Identification')

ldb_category_counts = df_port['littoraldb_id_conf'].value_counts()
pie_fig_recl = px.pie(values=ldb_category_counts, names=ldb_category_counts.index, color_discrete_sequence = [label_colors[x] for x in ldb_category_counts.index],
                      height = 500, width = 500)

fig.show(), pie_fig_recl.show()

(None, None)

In [61]:
label_colors = {'low': 'red', 'medium':'orange', 'high': 'green'}

country = [b'United States of America']
indices = np.where(ds_dr['country'].isin(country))[0]
df_us= ds_dr.isel(stations = indices).to_dataframe().reset_index()

df_us = df_us[(df_us['lat'].between(29, 34.6)) & (df_us['lon'].between(-82, -80))]

for dc in decode_cols:
    df_us[dc] = df_us[dc].str.decode("utf-8")

fig = px.scatter_mapbox(df_us, 
                    lat='lat', 
                    lon = 'lon',
                    color='nourishment_id_conf',
                    color_discrete_map= label_colors,
                    category_orders= {'reclamation_id_conf': list(label_colors.keys())},
                    hover_data= ['transect_id'],
                    center={'lat': df_us['lat'].mean(), 'lon': df_us['lon'].mean()},
                    mapbox_style='carto-positron', 
                    zoom=5.5,
                    width = 800,
                    height = 1000)


fig.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Nourishment Identification')

nour_category_counts = df_us['nourishment_id_conf'].value_counts()
pie_fig_recl = px.pie(values=nour_category_counts, names=nour_category_counts.index, color_discrete_sequence = [label_colors[x] for x in nour_category_counts.index],
                      height = 500, width = 500)

fig.show(), pie_fig_recl.show()

(None, None)

In [62]:
label_colors = {'low': 'red', 'medium':'orange', 'high': 'green', 'unknown': 'grey'}

country = [b'Netherlands']
indices = np.where(ds_dr['country'].isin(country))[0]
df_nl = ds_dr.isel(stations = indices).to_dataframe().reset_index()

decode_cols = ['seasonal_id_conf', 'littoraldb_id_conf', 'nourishment_id_conf', 'reclamation_id_conf', 'transect_id']
for dc in decode_cols:
    df_nl[dc] = df_nl[dc].str.decode("utf-8")
    
fig_scat_seas = px.scatter_mapbox(df_nl, 
                                lat='lat', 
                                lon = 'lon',
                                color='seasonal_id_conf',
                                color_discrete_map= label_colors,
                                category_orders= {'seasonal_id_conf': list(label_colors.keys())},
                                hover_data= ['transect_id'],
                                center={'lat': 52.5, 'lon': 5},
                                mapbox_style='carto-positron', 
                                zoom=6.5,
                                width = 800,
                                height = 1000)

fig_scat_seas.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Seasonality Identification')

seas_category_counts = df_nl['seasonal_id_conf'].value_counts()
seas_category_counts
pie_fig_seas = px.pie(values=seas_category_counts, names=seas_category_counts.index, color_discrete_sequence = [label_colors[x] for x in seas_category_counts.index],
                      height = 500, width = 500)

fig_scat_seas.show(), pie_fig_seas.show()

(None, None)

In [63]:
country = [b'Qatar', b'United Arab Emirates']
indices = np.where(ds_dr['country'].isin(country))[0]
df_q = ds_dr.isel(stations = indices).to_dataframe().reset_index()

for dc in decode_cols:
    df_q[dc] = df_q[dc].str.decode("utf-8")

fig = px.scatter_mapbox(df_q, 
                    lat='lat', 
                    lon = 'lon',
                    color='reclamation_id_conf',
                    color_discrete_map= label_colors,
                    category_orders= {'reclamation_id_conf': list(label_colors.keys())},
                    hover_data= ['transect_id'],
                    center={'lat': df_q['lat'].mean(), 'lon': df_q['lon'].mean()},
                    mapbox_style='carto-positron', 
                    zoom=5.5,
                    width = 800,
                    height = 1000)


fig.update_layout(legend=dict(
    title="Confidence Level",
    ),
    title = 'Reclamation Identification')

recl_category_counts = df_q['reclamation_id_conf'].value_counts()
pie_fig_recl = px.pie(values=recl_category_counts, names=recl_category_counts.index, color_discrete_sequence = [label_colors[x] for x in recl_category_counts.index],
                      height = 500, width = 500)

fig.show(), pie_fig_recl.show()

(None, None)